# Katz Centrality

In this notebook, we will compute the Katz centrality of each vertex in our test datase using both cuGraph and NetworkX. Additionally, NetworkX also contains a Numpy implementation that will used. The NetworkX and cuGraph processes will be interleaved so that each step can be compared.

| Author Credit |    Date    |  Update          | cuGraph Version |  Test Hardware |
| --------------|------------|------------------|-----------------|----------------|
| Brad Rees     | 10/15/2019 | created          | 0.14            | GV100, CUDA 10.2
| Brad Rees     | 08/16/2020 | tested / updated | 0.15.1 nightly  | RTX 3090 CUDA 11.4
| Don Acosta    | 07/05/2022 | tested / updated | 22.08 nightly   | DGX Tesla V100 CUDA 11.5
| Ralph Liu    | 07/26/2022 | updated | 22.08 nightly   | DGX Tesla V100 CUDA 11.5

## Introduction
Katz centrality is a measure of the relative importance of a vertex within the graph based on measuring the influence across the total number of walks between vertex pairs. 

<img src="https://latex.codecogs.com/gif.latex?C_{katz}(i)&space;=&space;\sum_{k=1}^{\infty}&space;\sum_{j=1}^{n}&space;\alpha&space;^k(A^k)_{ji}" title="C_{katz}(i) = \sum_{k=1}^{\infty} \sum_{j=1}^{n} \alpha ^k(A^k)_{ji}" />

See [Katz on Wikipedia](https://en.wikipedia.org/wiki/Katz_centrality) for more details on the algorithm.

To compute the Katz centrality scores for a graph in cuGraph we use:<br>
__df = cugraph.katz_centrality(G,alpha=0.1, max_iter=100, tol=1.0e-6, nstart=None, normalized=True)__

    G: cugraph.Graph object
    alpha: float, Attenuation factor. default is 0.1
    max_iter: int, The maximum number of iterations before an answer is returned. 
            This can be used to limit the execution time and do an early exit 
            before the solver reaches the convergence tolerance. If this value 
            is lower or equal to 0 cuGraph will use the default value, which is 100
    tol: float, Set the tolerance the approximation, this parameter should be a small 
            magnitude value. The lower the tolerance the better the approximation. If 
            this value is 0.0f, cuGraph will use the default value which is 0.00001. 
            Setting too small a tolerance can lead to non-convergence due to numerical 
            roundoff. Usually values between 0.01 and 0.00001 are acceptable.
    nstart: cuDataFrame, GPU Dataframe containing the initial guess for katz centrality. 
            Default is None
    normalized: bool, If True normalize the resulting katz centrality values.  
            Default is True

Returns:

    df: a cudf.DataFrame object with two columns:
        df['vertex']: The vertex identifier for the vertex
        df['katz_centrality']: The Katz centrality score for the vertex



The value of _alpha_ should be<br>
<img src="https://latex.codecogs.com/gif.latex?\alpha&space;<&space;\frac{1}{\lambda&space;_{max}&space;}" title="\alpha < \frac{1}{\lambda _{max} }" />
currently the user is responsible for setting alpha appropiatly.  

### _NOTICE_
There is a difference between how cuGraph and how NetworkX computes the Katz centrality score.  That difference leads to the scores not matching.  cuGraph does not currently support the 'beta' and 'weight' parameters as seen in the corresponding networkX call. The cuGraph implementation is based on a relaxed version of Katz defined by Foster with a reduced computational complexity of O(n+m)

    Foster, K.C., Muth, S.Q., Potterat, J.J. et al.
    Computational & Mathematical Organization Theory (2001) 7: 275.
    https://doi.org/10.1023/A:1013470632383


#### Some notes about vertex IDs...
* cuGraph will automatically renumber graphs to an internal format consisting of a contiguous series of integers starting from 0, and convert back to the original IDs when returning data to the caller. If the vertex IDs of the data are already a contiguous series of integers starting from 0, the auto-renumbering step can be skipped for faster graph creation times.
  * To skip auto-renumbering, set the `renumber` boolean arg to `False` when calling the appropriate graph creation API (eg. `G.from_cudf_edgelist(gdf_r, source='src', destination='dst', renumber=False)`).
  * For more advanced renumbering support, see the examples in `structure/renumber.ipynb` and `structure/renumber-2.ipynb`


### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


<img src="../../img/zachary_black_lines.png" width="35%"/>


Because the test data has vertex IDs starting at 1, the auto-renumber feature of cuGraph (mentioned above) will be used so the starting vertex ID is zero for maximum efficiency. The resulting data will then be auto-unrenumbered, making the entire renumbering process transparent to users.


### Importing needed Libraries

In [ ]:
# Import rapids libraries
import cugraph
import cudf

# Import a built-in dataset
from cugraph.experimental.datasets import karate

In [ ]:
# Import NetworkX libraries
import networkx as nx

### parameters
max_iter determines the number of iterations the algorithm will run to seek convergence
tol is the maximum difference that indicates convergence.
The algorithm will fail if it fails to converge within the tolerance (tol) after max_iter number of runs. This is often due to dataset characteristics.


In [ ]:
# define the parameters 
max_iter = 100  # The maximum number of iterations
tol = 0.00001   # tolerance

### Create a Graph 

In [ ]:
# Create a graph using the imported Dataset object
G = karate.get_graph(fetch=True)

In [ ]:
# compute degree and get the max
degree = G.degrees()
lamda = degree['out_degree'].max()

In [ ]:
print("The max degree is " + str(lamda))

### Call the Katz algorithm

In [ ]:
alpha = 1 / lamda

In [ ]:
# Call cugraph.katz_centrality to get the Katz scores
gdf_katz = cugraph.katz_centrality(G, alpha=alpha)

_It was that easy!_  

----

Let's now look at the results

In [ ]:
# let's sort the data and look at the top 5 vertices
gdf_katz.sort_values(by='katz_centrality', ascending=False).head(5)

---

## Now compute using NetworkX

In [ ]:
# Read the data, this also created a NetworkX Graph 
datafile = "../../data/karate-data.csv"
file = open(datafile, 'rb')
Gnx = nx.read_edgelist(file)

In [ ]:
k_nx = nx.katz_centrality(Gnx, alpha=alpha, max_iter=max_iter, tol=tol)

In [ ]:
k_nx_s = sorted(((value, key) for (key,value) in k_nx.items()), reverse=True)

In [ ]:
k_nx_s[:5]

As mentioned, the scores are different but the ranking is the same.

In [ ]:
# The Numpy version
k_nx_mp = nx.katz_centrality_numpy(Gnx, alpha=alpha)

In [ ]:
sorted(((value, key) for (key,value) in k_nx_mp.items()), reverse=True)[:5]

___
Copyright (c) 2019-2022, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___